In [96]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.request
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

In [172]:
def find_links_to_brand_pages(URL = "https://www.endclothing.com/gb/brands"):

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    letters = ['#', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    driver.get(URL)
    brand_links = []

    for letter in letters:
        id = 'letter-' + letter
        try:
            element = driver.find_element(By.ID, id)
            brands = element.find_elements(By.TAG_NAME, 'a')

            for brand in brands:
                link = brand.get_attribute('href')
                brand_links.append(link)
        except:
            print('No brands under: ', letter)

    return brand_links

In [175]:
def find_links_to_all_items_by_band(brand_links: list[str]):

    clothing_link_dictionary = {}

    for URL in brand_links:

        clothing_links = []

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(URL)

        try:
            a_s = driver.find_elements(By.TAG_NAME, "a")
        except:
            print("Error with link: ", URL)

        for a in a_s:

            try:
                if "styles__ProductCardSC-sc-zanqkz-2 bsgZRO" in a.get_attribute(name="class"):
                    clothing_links.append(a.get_attribute("href"))
            except:
                continue

        clothing_link_dictionary[URL.split("/")[-1]] = clothing_links

    return clothing_link_dictionary

In [176]:
brand_links = find_links_to_brand_pages()
clothing_links = find_links_to_all_items_by_band(brand_links=brand_links)

No brands under:  x
No brands under:  z


Number of brands: 606

Number of items of clothing: 21782

In [144]:
## Find the urls to all brands on END
base_url = 'https://www.endclothing.com'

def find_links_to_brands():

    url = base_url + '/gb/brands'
    response = requests.get(url)

    # Parse the HTML content with Beautiful Soup
    soup = BeautifulSoup(response.content, 'lxml')  # or 'html.parser'

    # To find all elements with that class
    brands_element = soup.find('ul', class_='BrandsList__CompleteBrandsListSC-sc-119n13o-0 iajMvd')

    # Find the elements for each brand
    brands = brands_element.find_all('li', class_='LetterBrandsSection__BrandListItemSC-sc-1hk2oim-1 gYwYqb')

    # Find brand links
    brand_links = []
    for brand in brands:
        brand_link_append = brand.find('a').get('href')
        brand_links.append(base_url + brand_link_append)

    return brand_links

def find_links_to_items_for_brand(brand_link: str):
    
    # Get html of page through selenium
    try:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(options=options)
        driver.get(brand_link)
        page = driver.page_source
        driver.quit()
        soup = BeautifulSoup(page, 'lxml')

        # Find all the items
        items = soup.find_all('a', class_='Link__InnerLink-sc-e4qf6g-0 iJLkPn styles__ProductCardSC-sc-zanqkz-2 bsgZRO')

        # Get urls for items in the brand
        item_links = []
        for item in items:
            item_link_append = item.get('href')
            item_links.append(base_url + item_link_append)

    except:
        print('Error with: ' + brand_link)
        item_links = []

    return item_links

def find_links_to_all_items():

    # Get links to all brands
    brand_links = find_links_to_brands()

    items = {}
    for brand_link in brand_links:

        print(brand_link)

        # Get name of brand from url
        brand_name = brand_link.split('/')[-1]

        # Get links to all items under that brand
        brand_items = find_links_to_items_for_brand(brand_link)

        # Add these to the item dictionary with the brand name as the key
        items[brand_name] = brand_items

    return items

def find_info_on_item(item_url: str):

    print(item_url)

    # Get page source
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=options)
    driver.get(item_url)
    page = driver.page_source
    driver.quit()
    soup = BeautifulSoup(page, 'lxml')

    # Get the product name
    product_name = soup.find('span', class_='ProductDetails__ProductTitleSC-sc-1fqzeck-2 lkbOYb').text

    # Get the price
    price = soup.find('span', class_='ProductDetails__FinalPriceSC-sc-1fqzeck-6 egKCfJ').text

    # Get sizes available
    sizes_available = [el.text for el in soup.find_all('SizeSelector__SizeButtonSC-sc-1rwf8s7-3 biAAmz')]

    # Get text description of item
    description_element = soup.find('div', class_='styles__TabWrapper-sc-btdnui-1 iJerob')
    text_description = description_element.find('p').text
    bullet_point_descriptions = [el.text for el in description_element.find_all('li')]

    # Get image links
    image_ribbon = soup.find('div', class_='ProductRibbon__SmallRibbonSC-sc-cvb65h-1 uJOQj')
    image_elements = image_ribbon.find_all('img')

    image_links = []
    for image_element in image_elements:
        image_links.append(image_element.get('src'))

    item_dictionary = {
        'product_name': product_name,
        'price': price,
        'sizes': sizes_available,
        'text_description': text_description,
        'bullet_point_description': bullet_point_descriptions,
        'image_links': image_links
    }

    return item_dictionary

def find_item_data(item_links: list[str]):

    item_datas = {}
    for item_link in item_links:
        try:
            item_data = find_info_on_item(item_link)
            item_datas[item_data['product_name']] = item_data
        except:
            print('Error with: ' + item_link)

    return item_datas

def find_all_data_on_items():

    items_links = find_links_to_all_items()

    items_data = {}
    for brand in items_links.keys():
        items_data[brand] = find_item_data(items_links[brand])

    return items_data

In [116]:
items = find_links_to_all_items()

https://www.endclothing.com/gb/brands/1017-alyx-9sm
https://www.endclothing.com/gb/brands/3-paradis
https://www.endclothing.com/gb/brands/424
https://www.endclothing.com/gb/brands/66-north
https://www.endclothing.com/gb/brands/olaf
https://www.endclothing.com/gb/brands/a-bathing-ape
https://www.endclothing.com/gb/brands/a-bathing-ape-eyewear
https://www.endclothing.com/gb/brands/a-bathing-ape-kids
https://www.endclothing.com/gb/brands/a-kind-of-guise
https://www.endclothing.com/gb/brands/a-cold-wall
https://www.endclothing.com/gb/brands/a-p-c
https://www.endclothing.com/gb/brands/aape-by-a-bathing-ape
https://www.endclothing.com/gb/brands/about-blank
https://www.endclothing.com/gb/brands/abrams-chronicle
https://www.endclothing.com/gb/brands/ace-tate
https://www.endclothing.com/gb/brands/acne-studios
https://www.endclothing.com/gb/brands/acne-studios-mini
https://www.endclothing.com/gb/brands/acronym
https://www.endclothing.com/gb/brands/adidas
https://www.endclothing.com/gb/brands/adi

In [145]:
data = {}
for brand in items.keys():
    data[brand] = find_item_data(items[brand])

https://www.endclothing.com/gb/1017-alyx-9sm-circle-logo-popover-hoodie-aausw0177fa01-blk0001.html
https://www.endclothing.com/gb/1017-alyx-9sm-side-zip-camo-pant-aampa0455fa01-mty0001.html
https://www.endclothing.com/gb/1017-alyx-9sm-x-blackmeans-denim-jacket-azmou0226fa05-blk0003.html
https://www.endclothing.com/gb/1017-alyx-9sm-oversized-camo-nylon-bomber-jacket-aamou0473fa02-mty0001.html
https://www.endclothing.com/gb/1017-alyx-9sm-nylon-logo-varsity-jacket-aamou0481fa01-blk0001.html
https://www.endclothing.com/gb/1017-alyx-9sm-buckle-puffer-vest-x-aauou0443fa01-blk0001.html
https://www.endclothing.com/gb/1017-alyx-9sm-buckle-canvas-jacket-aauou0470fa01-blk0003.html
https://www.endclothing.com/gb/1017-alyx-9sm-paraboot-aaubo0087fa01-blk0001.html
https://www.endclothing.com/gb/1017-alyx-9sm-work-boot-aaubo0075fa01-blk0001.html
https://www.endclothing.com/gb/1017-alyx-9sm-buckle-short-sleeve-shirt-aaush0102fa04-blk0001.html
Error with: https://www.endclothing.com/gb/1017-alyx-9sm-buc

In [147]:
import json

with open("data.json", "w") as outfile: 
    json.dump(data, outfile)